In [1]:
import math
import random
import torch
import torchvision
import time 
from torchvision import datasets ,transforms
from torch.utils.data import DataLoader
from torch import nn 
from torch.nn.functional import relu ,softmax 
import copy 

In [2]:
data_transform_train=transforms.Compose ([
     transforms.ToTensor(),
                                        
                                          
])
data_transform_test=transforms.Compose ([transforms .ToTensor(),
                                          
])
data_transform={'train':data_transform_train,
                'val':data_transform_test
                }

In [3]:
mnist_data_train=datasets.MNIST('MNIST_data',train=True,transform=data_transform['train'] ,download=True)
mnist_data_test=datasets.MNIST('MNIST_data',train=False ,download=True ,transform=data_transform["val"])
dataloader={'train':DataLoader(mnist_data_train,shuffle=True ,batch_size=16),
            'val':DataLoader(mnist_data_test,shuffle=True,batch_size=16)}
dataset_sizes={'train':len(mnist_data_train),
               'val':len(mnist_data_test)}
device=torch.device( "cpu")
print(dataset_sizes["train"])
print (dataset_sizes["val"])
#for images,labels in dataloader["train"]:
  #print(images.shape ,type(images ),labels)


60000
10000


In [4]:

class Model(torch.nn.Module):
    def __init__(self):
        super (Model,self ).__init__()
        self .linear1=nn.Linear (28*28,3000)
        self.linear2=nn.Linear (3000,10)
        
    def Forward (self,x,mask,p):
        #feed forward function 
        x=x.view(-1,28*28)
        act1=relu(self.linear1(x))
        act1_masked =self.masking (act1,mask,p)
        act2=softmax(self.linear2(act1_masked ))
        return act2
    
    
    
    def masking (self,act1,mask,p):
        if (self.training ==True ):
            return ((act1*mask)/p)
        else :
            return (act1)
        



  
  
  


      

In [5]:
model=Model()#creating the object of the class
model.to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.001,momentum=0.9)

In [6]:
def train_model(model, criterion, optimizer,mask,p, num_epochs=25):
  # the mask has to be specified 

      since = time.time()

      best_model_wts = copy.deepcopy(model.state_dict())
      best_acc = 0.0
      losses=[]
      accuracies=[]

      for epoch in range(num_epochs):
          print('Epoch {}/{}'.format(epoch, num_epochs - 1))
          print('-' * 10)

          # Each epoch has a training and validation phase
          for phase in ['train', 'val']:
              if phase == 'train':
                  
                  model.train()  # Set model to training mode
              else:

                  model.eval()   # Set model to evaluate mode

              running_loss = 0.0
              running_corrects = 0

              # Iterate over data.
              for inputs, labels in dataloader[phase]:
                inputs=inputs.to(device)
                labels=labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs= model.Forward(inputs,mask,p)

                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)# backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        #print (model.linear1.weight.grad)
                        #print (model.linear2.weight.grad)

                  # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                  

              epoch_loss = running_loss / dataset_sizes[phase]
              epoch_acc = running_corrects.double() / dataset_sizes[phase]
            

              print('{} Loss: {:.4f} Acc: {:.4f} '.format(
                  phase, epoch_loss, epoch_acc))

              # deep copy the model
              if phase == 'val' and epoch_acc > best_acc:
                  best_acc = epoch_acc
                  best_model_wts = copy.deepcopy(model.state_dict())
              if phase=='train':
                losses.append(epoch_loss)
                accuracies.append(epoch_acc)


          

      time_elapsed = time.time() - since
      print('Training complete in {:.0f}m {:.0f}s'.format(
          time_elapsed // 60, time_elapsed % 60))
      print('Best val Acc: {:4f}'.format(best_acc))

      # load best model weights
      model.load_state_dict(best_model_wts)
      return model,losses,accuracies

In [7]:
class Population:
    
    def __init__(self,m,num,maskLength):
        # constructor for initialising the population list
        #list of DNA objects
        self.population=[]
        #muation rate for mutation
        self.mutation_rate=m
        #maximum number of entities in the population
        self.popmax=num

        self.maskLength=maskLength
        for i in range (num):
            #creating a dna object
            #an initial random population created 
            dna =DNA(self.maskLength)
            self.population.append (dna)
      
        self.matingPool=[]
    




    def calcFitness (self,model):
        # going through all the entities of population 
        #finding fitness of all population entities 
        for i in range(0,self.popmax):
            self.population[i].fitness (model)




    def naturalSelection(self):
        self.matingPool=[]
        maxFitness=0
        for i in range (self.popmax):
            # moving throught the entire population 
            if (self.population[i].fit>maxFitness):
                maxFitness=self.population[i].fit
       
        # max Fitness has the maximum loss score of the entire population  
        for i in range (self.popmax ):
        # iterating through the all inviduals of the population
            n=self.Mymap(self.population[i].fit,0,maxFitness,0,1)
        
            n=math.floor(n*100)
            
            for j in range (n):
                #creating mating pool
                self.matingPool.append (self.population[i])




    def Mymap(self,num,prevlow,prevhigh,nextlow,nexthigh):
        
        prevrange =float((num-prevlow)/(prevhigh-prevlow))
        return nextlow+(nexthigh-nextlow)*prevrange



    def   generate (self):
        for i in range (self.popmax ):
            index_1=math.floor(random.randint  (0,len(self.matingPool)-1))
            index_2=math.floor (random.randint (0,len(self.matingPool)-1))
            parent1=self.matingPool[index_1]
            parent2=self.matingPool[index_2]
            child=parent1.crossover(parent2)
            child.mutate(self.mutation_rate)
            self.population[i]=child 


    def fittest(self):
        #returns the fiitest individual mask of the population 
        #also returns the keeping probability of the fittest mask 
        fittest=self.population[0]
        for i  in range (self.popmax):
            if (fittest.fit<self.population[i].fit):
                fittest=self.population[i]
        return fittest,fittest.keep_prob()








In [8]:



class DNA:
    
    
    def __init__(self,maskLength):
        #constructor for the creation of the mask as a gene object 
        self.maskLength=maskLength
        #creation of mask 
        self.gene=torch.bernoulli(torch.empty(1,maskLength).uniform_(0,1))
        self.fit=0




    def keep_prob (self):

        num_one =0
        for i in range (self.maskLength):
            if (self.gene[0,i]==1):
                num_one=num_one+1
        return float(num_one/self.maskLength)




    def fitness(self,model):
        # finding the fitness of a particular mask
        #accuracy of all training set is the fitness in one epoch
        #putting model in train mode 
        running_loss=0
        model.train()
        
        for inputs,labels in dataloader ['train']:
            inputs=inputs.to(device)
            labels=labels.to(device )
            outputs=model.Forward(inputs,self.gene,self.keep_prob())
            _,preds=torch.max(outputs,1)
            loss=criterion (outputs,labels)
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / dataset_sizes['train']
        
        self.fit=epoch_loss
        return epoch_loss











    def crossover (self,parent2):
        #one parent is the passed in the argument 
        #another parent is the one from which this function is called 
        #another parent is self.gene
        child =DNA(self.maskLength)
        midpoint =random .randint (0,self.maskLength-1)
        for i in range (0,self.maskLength):
            if (i>midpoint):
                child.gene [0,i]=self.gene[0,i]
            else :
                child.gene [0,i]=parent2.gene[0,i]
        
        return child 

    def mutate(self,mutation_rate):
        #randomly activate some of the nodes  
        #mutate some of the genes 
        for i in range (self.maskLength):
            if (random.randint (0,99)<=mutation_rate*100):
                self.gene[0,i]=1











    

In [9]:
mutation_rate =0.15
max_population=40
maskLength=3000
model=Model()#creating the object of the class
model.to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.001,momentum=0.9)



In [ ]:
#control block controls the epochs and the generations of mask
#step 1 an object of the population class randomly generating the first population 
#step2 :calculate fitness of each entitiy of the population 
#step3: creates a mating pool of the population based on the worst two performing parent 
#step 4 :fittest mask of the generating along with keep_prob found 
#step 5: if 0th ,10th ,20th, the epochs starts training on the worst performing mask /other wise new generation is created 

epochgens=0
population =Population(mutation_rate,max_population,maskLength)

while (epochgens<300):
    print ('Epoch generations (',epochgens,'/300)',end=' :')
    population .calcFitness(model)
    population.naturalSelection()
    fittestmask,p=population .fittest()
    loss=fittestmask.fitness(model)
    print ("loss (fittest mask)",loss,end='\n')
    if (epochgens%50==0):
        model,losses,accuracies=train_model(model,criterion,optimizer,fittestmask.gene,p,25)
    population.generate()
    epochgens+=1



Epoch generations ( 0 /300) :

/home/cmater/anaconda3/envs/cmater/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


loss (fittest mask) 2.3041300769170125
Epoch 0/24
----------
train Loss: 1.8198 Acc: 0.7152 
val Loss: 1.7668 Acc: 0.8323 
Epoch 1/24
----------
train Loss: 1.6458 Acc: 0.8338 
val Loss: 1.7058 Acc: 0.8470 
Epoch 2/24
----------
train Loss: 1.6010 Acc: 0.8807 
val Loss: 1.6487 Acc: 0.9142 
Epoch 3/24
----------
train Loss: 1.5628 Acc: 0.9154 
val Loss: 1.6193 Acc: 0.9243 
Epoch 4/24
----------
train Loss: 1.5519 Acc: 0.9225 
val Loss: 1.6051 Acc: 0.9286 
Epoch 5/24
----------
train Loss: 1.5447 Acc: 0.9278 
val Loss: 1.5946 Acc: 0.9319 
Epoch 6/24
----------
train Loss: 1.5391 Acc: 0.9331 
val Loss: 1.5862 Acc: 0.9357 
Epoch 7/24
----------
train Loss: 1.5345 Acc: 0.9365 
val Loss: 1.5790 Acc: 0.9392 
Epoch 8/24
----------
train Loss: 1.5304 Acc: 0.9401 
val Loss: 1.5744 Acc: 0.9423 
Epoch 9/24
----------
train Loss: 1.5271 Acc: 0.9428 
val Loss: 1.5690 Acc: 0.9433 
Epoch 10/24
----------
train Loss: 1.5239 Acc: 0.9456 
val Loss: 1.5669 Acc: 0.9446 
Epoch 11/24
----------
train Loss: 1

Epoch generations ( 57 /300) :loss (fittest mask) 1.4913756962458293
Epoch generations ( 58 /300) :loss (fittest mask) 1.4913756969769796
Epoch generations ( 59 /300) :loss (fittest mask) 1.4913756957372029
Epoch generations ( 60 /300) :loss (fittest mask) 1.4913756986618043
Epoch generations ( 61 /300) :loss (fittest mask) 1.491375697072347
Epoch generations ( 62 /300) :loss (fittest mask) 1.4913756990432738
Epoch generations ( 63 /300) :loss (fittest mask) 1.4913756976445516
Epoch generations ( 64 /300) :loss (fittest mask) 1.4913756992022196
Epoch generations ( 65 /300) :loss (fittest mask) 1.4913756972312928
Epoch generations ( 66 /300) :loss (fittest mask) 1.4913756965955098
Epoch generations ( 67 /300) :loss (fittest mask) 1.4913756948471069
Epoch generations ( 68 /300) :loss (fittest mask) 1.4913756981531778
Epoch generations ( 69 /300) :loss (fittest mask) 1.4913756994565328
Epoch generations ( 70 /300) :loss (fittest mask) 1.4913756985982258
Epoch generations ( 71 /300) :loss 

Epoch generations ( 144 /300) :loss (fittest mask) 1.4836777241071066
Epoch generations ( 145 /300) :loss (fittest mask) 1.4836777256011964
Epoch generations ( 146 /300) :loss (fittest mask) 1.4836777249654134
Epoch generations ( 147 /300) :loss (fittest mask) 1.4836777243614196
Epoch generations ( 148 /300) :loss (fittest mask) 1.4836777240435282
Epoch generations ( 149 /300) :loss (fittest mask) 1.4836777215003967
Epoch generations ( 150 /300) :loss (fittest mask) 1.4836777216911317
Epoch 0/24
----------
train Loss: 1.4838 Acc: 0.9830 
val Loss: 1.4920 Acc: 0.9735 
Epoch 1/24
----------
train Loss: 1.4836 Acc: 0.9832 
val Loss: 1.4919 Acc: 0.9739 
Epoch 2/24
----------
train Loss: 1.4833 Acc: 0.9833 
val Loss: 1.4916 Acc: 0.9746 
Epoch 3/24
----------
train Loss: 1.4831 Acc: 0.9837 
val Loss: 1.4915 Acc: 0.9748 
Epoch 4/24
----------
train Loss: 1.4829 Acc: 0.9837 
val Loss: 1.4915 Acc: 0.9740 
Epoch 5/24
----------
train Loss: 1.4826 Acc: 0.9839 
val Loss: 1.4913 Acc: 0.9748 
Epoch 